<a href="https://colab.research.google.com/github/bird0401/Instance_level_recognition/blob/main/02_resnet_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning==0.7.1

     |████████████████████████████████| 6.0 MB 8.1 MB/s 
     |████████████████████████████████| 829 kB 47.1 MB/s 
  Created wheel for pytorch-lightning: filename=pytorch_lightning-0.7.1-py3-none-any.whl size=145329 sha256=de117fd4f7f86e8425202a3d211a6b48c52a99ef1490a40be130640a0b3e10a6
  Stored in directory: /root/.cache/pip/wheels/a5/c0/6c/ed64904da20814878f410c520ae61c062c6d7e93bf5c27dcd4
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=6aaa02bc84e9064ee75e23fcf90013d90e18002863cc17a4ca4795121f976b7d
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built pytorch-lightning future
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [2]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer

import torch,torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.models import resnet18
import torch.utils.data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split

from collections import defaultdict
import os
import random
from tqdm import tqdm
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset

In [4]:
categories=["dog","car2","uspresident","uspresident_02","tower","tower_02"]
category=categories[0]+"_100"

In [5]:
# !cp /content/drive/MyDrive/instance_level_recognition/dog_2.tar.gz .
!cp /content/drive/MyDrive/instance_level_recognition/imgs_{category}.tar.gz .

In [6]:
!tar -zxvf imgs_{category}.tar.gz

ストリーミング出力は最後の 5000 行に切り捨てられました。
imgs_dog/Q39295_074.jpg
imgs_dog/Q38287_011.jpg
imgs_dog/Q29099_085.jpg
imgs_dog/Q39285_035.jpg
imgs_dog/Q38280_150.jpg
imgs_dog/Q37629_111.jpg
imgs_dog/Q39026_019.jpg
imgs_dog/Q38280_098.jpg
imgs_dog/Q37888_009.jpg
imgs_dog/Q5408020_019.jpg
imgs_dog/Q38080_027.JPG
imgs_dog/Q39295_117.jpg
imgs_dog/Q37710_150.jpg
imgs_dog/Q38280_056.jpg
imgs_dog/Q24702349_001.jpg
imgs_dog/Q37710_258.jpg
imgs_dog/Q29099_231.jpg
imgs_dog/Q38358_098.jpg
imgs_dog/Q38287_037.jpg
imgs_dog/Q39058_036.jpg
imgs_dog/Q37888_053.JPG
imgs_dog/Q38280_421.JPG
imgs_dog/Q39295_180.jpg
imgs_dog/Q37688_003.jpg
imgs_dog/Q37688_078.jpg
imgs_dog/Q29114_036.JPG
imgs_dog/Q38686_244.jpg
imgs_dog/Q38686_045.jpg
imgs_dog/Q38152_002.jpg
imgs_dog/Q38080_010.jpg
imgs_dog/Q38287_211.jpg
imgs_dog/Q21205_034.jpg
imgs_dog/Q37617_058.jpg
imgs_dog/Q37750_040.jpg
imgs_dog/Q37617_032.jpg
imgs_dog/Q29099_238.jpg
imgs_dog/Q38280_422.JPG
imgs_dog/Q38280_475.JPG
imgs_dog/Q39058_115.jpg
imgs_dog/Q38686_304.jpg
img

In [20]:
!ls imgs_{category}|wc -l

5953


In [21]:
!rm -rf /content/{category}.tar.gz

In [22]:
# # 224×224で正規化した画像に変換

class ImageTranform():
  def __init__(self):
    self.data_transform=transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
  def __call__(self,img):
    return self.data_transform(img)

In [23]:
# transform=ImageTranform()
# img=Image.open("images/Q17504_001.jpg")
# img=transform(img)
# # img=img.numpy().transpose((1,2,0)) # imshow用に次元入れ替え
# img=np.clip(img,0,1)
# plt.imshow(img)
# plt.show()
# print(img.shape)
# print(img.dtype)

In [24]:
# 実行後のファイルはすでに保存してあるのでコメントアウト

# # image_size=32 #試し用
# image_size=224 

# image_shape=(image_size,image_size,3)

# ファイルの入出力用にglobをインポート
import glob

image_dir = "imgs_"+category
paths=glob.glob(image_dir+"/*")

## 準備

# q_files = defaultdict(list)
imgs,qids=[],[]
transform=ImageTranform()

for path in tqdm(paths):
  try:
    # img=np.array(Image.open(path).convert("RGB").resize((image_size,image_size)))
    img=Image.open(path)
    img=transform(img)
    img=img.numpy()
    img=np.clip(img,0,1)
  except Exception as e:
    print(e)
  else:
    ## キーと画像のリストを保存
    filename=path.split("/")[1]
    qid = filename.split("_")[0][1:]
    qids.append(qid)
    imgs.append(img)
    ## キーから画像を取得できるようにする
    # q_files[qid].append(img)

  1%|▏         | 85/5953 [00:09<09:13, 10.59it/s]

output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]


  2%|▏         | 112/5953 [00:12<09:10, 10.60it/s]

cannot identify image file 'imgs_dog_100/Q21206_042.webm'


  2%|▏         | 144/5953 [00:15<07:03, 13.73it/s]

...

 98%|█████████▊| 5845/5953 [10:05<00:14,  7.43it/s]

The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0


 99%|█████████▉| 5905/5953 [10:11<00:03, 15.44it/s]

cannot identify image file 'imgs_dog_100/Q38280_202.ogv'


100%|██████████| 5953/5953 [10:17<00:00,  9.65it/s]


In [25]:
# #  pickleで保存
with open('qids_'+category+'.binaryfile','wb') as f:
  pickle.dump(qids,f)

with open('imgs_'+category+'.binaryfile','wb') as f:
  pickle.dump(imgs,f)

In [26]:
# # 保存したファイルをdriveにコピー
!cp /content/{'qids_'+category+'.binaryfile'} /content/drive/MyDrive/instance_level_recognition
!cp /content/{'imgs_'+category+'.binaryfile'} /content/drive/MyDrive/instance_level_recognition

In [17]:
# 保存したファイルをdriveから取ってくる
!cp /content/drive/MyDrive/instance_level_recognition/{'qids_'+category+'.binaryfile'} .
# !cp /content/drive/MyDrive/instance_level_recognition/imgs.binaryfile .

In [18]:
# pickleからロード
with open('/content/qids_'+category+'.binaryfile','rb') as f:
  qids=pickle.load(f)
# with open('/content/imgs.binaryfile','rb') as f:
#   imgs=pickle.load(f)

In [27]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(qids)
y_labels=le.transform(qids)

In [28]:
len(qids)

5745

In [29]:
y_labels

array([41, 80, 53, ..., 76, 69, 22])

In [30]:
n_classes=len(set(y_labels.tolist()))

In [31]:
imgs=torch.tensor(imgs,dtype=torch.float32)
y_labels=torch.from_numpy(y_labels.astype(np.uint8)).clone()

# X_train, X_valid, X_test = X_train/255.0, X_valid/255.0, X_test/255.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


In [32]:
len(y_labels)

5745

In [33]:
#  pickleで保存
with open('y_labels_'+category+'.binaryfile','wb') as f:
  pickle.dump(y_labels,f)

with open('imgs_'+category+'.binaryfile','wb') as f:
  pickle.dump(imgs,f)

In [34]:
# 保存したファイルをdriveにコピー
!cp /content/{'y_labels_'+category+'.binaryfile'} /content/drive/MyDrive/instance_level_recognition
!cp /content/{'imgs_'+category+'.binaryfile'} /content/drive/MyDrive/instance_level_recognition

In [21]:
# # 保存したファイルをdriveから取ってくる
# !cp /content/drive/MyDrive/instance_level_recognition/{'y_labels_'+category+'.binaryfile'} .
# !cp /content/drive/MyDrive/instance_level_recognition/'imgs_'+category+'.binaryfile' .

cp: cannot stat '/content/drive/MyDrive/instance_level_recognition/y_labels_dog_100.binaryfile': No such file or directory
cp: cannot stat '/content/drive/MyDrive/instance_level_recognition/imgs_+category+.binaryfile': No such file or directory


In [22]:
# # pickleからロード
# with open('y_labels_'+category+'.binaryfile','rb') as f:
#   y_labels=pickle.load(f)
# with open('imgs_'+category+'.binaryfile','rb') as f:
#   imgs=pickle.load(f)

In [35]:
# 統合
dataset=torch.utils.data.TensorDataset(imgs,y_labels)
dataset

In [40]:
n_train = int(len(dataset) * 0.6)
n_val = int((len(dataset) - n_train) * 0.5)
n_test = len(dataset) - n_train - n_val
n_train, n_val, n_test

(3447, 1149, 1149)

In [41]:
torch.manual_seed(0)
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])
# len(X_train),len(X_valid),len(X_test)

# Resnet18

In [42]:
class TrainNet(pl.LightningModule):
  def train_dataloader(self):
    return torch.utils.data.DataLoader(train,self.batch_size,shuffle=True)
  def training_step(self,batch,batch_nb):
    x,t=batch
    y=self.forward(x)
    loss=self.lossfun(y,t)
    results={"loss":loss}
    return results

class ValidationNet(pl.LightningModule):
  def val_dataloader(self):
        return torch.utils.data.DataLoader(val, self.batch_size)
  def validation_step(self,batch,batch_nb):
    x,t=batch
    y=self.forward(x)
    loss=self.lossfun(y,t)
    y_label=torch.argmax(y,dim=1)
    acc=torch.sum(t==y_label)/len(t)
    results={"val_loss":loss,"val_acc":acc}
    return results
  def validation_end(self,outputs):
    avg_loss=torch.stack([x["val_loss"] for x in outputs]).mean()
    avg_acc=torch.stack([x["val_acc"] for x in outputs]).mean()
    results={"val_loss":avg_loss,"val_acc":avg_acc}
    return results

class TestNet(pl.LightningModule):
    def test_dataloader(self):
        return torch.utils.data.DataLoader(test, self.batch_size)

    def test_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'test_loss': loss, 'test_acc': acc}
        return results

    def test_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        results = {'test_loss': avg_loss, 'test_acc': avg_acc}
        return results

In [43]:
resnet=resnet18(pretrained=True)

class Net(TrainNet,ValidationNet, TestNet):
  def __init__(self,batch_size=256):
    super().__init__()
    self.batch_size=batch_size
    self.conv=resnet
    self.fc=nn.Linear(1000,n_classes)
    for param in self.conv.parameters():
      param.requires_grad=False
      
  def lossfun(self, y, t):
      return F.cross_entropy(y, t)

  def configure_optimizers(self):
      return torch.optim.SGD(self.parameters(), lr=0.01)

  def forward(self, x):
      x = self.conv(x)
      x = self.fc(x)
      return x

In [44]:
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False
torch.manual_seed(0)

net=Net()
trainer=Trainer(gpus=1,max_epochs=30,batch_size=1024)
trainer.fit(net)

Validation sanity check:   0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Validating:   0%|          | 0/5 [00:00<?, ?it/s]

Validating:   0%|          | 0/5 [00:00<?, ?it/s]

Validating:   0%|          | 0/5 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/5 [00:00<?, ?it/s]

Validating:   0%|          | 0/5 [00:00<?, ?it/s]

Validating:   0%|          | 0/5 [00:00<?, ?it/s]

Validating:   0%|          | 0/5 [00:00<?, ?it/s]

1

# Results

In [45]:
trainer.test()
trainer.callback_metrics

Testing:   0%|          | 0/5 [00:00<?, ?it/s]

----------------------------------------------------------------------------------------------------
TEST RESULTS
{}
----------------------------------------------------------------------------------------------------


{'epoch': 29,
 'loss': 1.5061700344085693,
 'test_acc': 0.38007500767707825,
 'test_loss': 2.528122663497925,
 'val_acc': 0.4136687219142914,
 'val_loss': 2.4690754413604736}